# **LangChain ile Kişiye Özel Chatbot - PDF'lerinizle konuşun!**


**Notebook'u hazırlayan: [Lokman Baturay Efe](https://www.linkedin.com/in/lokmanefe/)**


0. İndirmeler, Kütüphaneler and API Anahtarları
1. PDF'leri yükleme ve LangChain ile parçalara ayırma
2. Metinleri gömme (embedding) ve gömme sonuçlarını depolama
3. Erişim (retrieval) fonksiyonu oluşturma
4. Sohbet hafızalı bir sohbet botu oluşturma
5. Tüm işlemi Gradio ile daha kullanılabilir hale getirme

**Notebook hazırlanırken [Liam Ottley](https://youtube.com/@LiamOttley)'in hazırladığı [notebook](https://colab.research.google.com/drive/1OZpmLgd5D_qmjTnL5AsD1_ZJDdb7LQZI?usp=sharing) referans alınmıştır.**








# 0. İndirmeler, Kütüphaneler and API Anahtarları

---



In [ ]:
!pip install pip==24.0

In [ ]:
!pip install -q pypdf pandas matplotlib tiktoken transformers faiss-cpu langchain-community langchain-google-genai textract==1.6.5 gradio google-cloud-aiplatform[tokenization]

In [5]:
import os
import textract
import pandas as pd
import matplotlib.pyplot as plt
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from vertexai.preview import tokenization
import gradio as gr

In [6]:
os.environ["GEMINI_API_KEY"] = "API anahtarınızı girin."

file_name = "PDF dosya ismini girin."

# 1. PDF'leri yükleme ve LangChain ile parçalara ayırma

---



In [ ]:
# Öncelikle kullanacağımız PDF dosyalarını Colab'in dosyalar kısmına eklememiz gerekiyor!

# PDF'i hafızaya yüklüyoruz.
loader = PyPDFLoader(f"./{file_name}.pdf")

# PDF'i okuyup sayfalara ayırıyoruz.
pages = loader.load_and_split()
print(pages[0])

In [10]:
chunks = pages
# Adım 1: PDF dosyasını metne çeviriyoruz.
doc = textract.process(f"./{file_name}.pdf")

# Adım 2: Hatalara engel olması için .txt olarak kaydedip tekrar okuyoruz.
with open(f"./{file_name}.txt", 'w') as f:
    f.write(doc.decode('utf-8'))

with open(f"./{file_name}.txt", 'r') as f:
    text = f.read()

# Adım 3: Tokenleri saymak için bir fonksiyon oluşturuyoruz.
tokenizer = tokenization.get_tokenizer_for_model("gemini-1.5-flash")

def count_tokens(text: str) -> int:
    result = tokenizer.count_tokens(text)
    return result.total_tokens

# Adım 4: Metinleri parçalara ayırıyoruz.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [ ]:
# Oluşan sonuçlar her biri yaklaşık 500 token veya 500'den daha az token
# içeren LangChain Document nesneleridir. (RecursiveCharacterTextSplitter yüzünden
# token sayısı 500'den fazla da olabilir, eğer bağlamı koparmak kötüyse 500'ü aşabilir)
type(chunks[0])

In [ ]:
# Parçalara ayırma işleminin başarılı olup olmadığını anlamak için bir görselleştirme yapalım.

# Her bir parçadaki token sayısının bir listesini oluşturalım.
token_counts = [count_tokens(chunk.page_content) for chunk in chunks]

# Oluşturduğumuz listeden bir veri tablosu oluşturalım.
df = pd.DataFrame({'Token Sayisi': token_counts})

# Token sayısı dağılımını incelemek için bir histogram oluşturalım.
df.hist(bins=40, )

# Oluşturduğumuz grafiği gösterelim.
plt.show()

# 2. Metinleri gömme (embedding) ve gömme sonuçlarını depolama

---



In [13]:
# Embedding (metin gömme) modelini çağıralım.
embeddings = GoogleGenerativeAIEmbeddings(google_api_key=os.environ.get("GEMINI_API_KEY"), model="models/embedding-001")

# Oluşturduğumuz embeddinglerden bir vektör veritabanı oluşturalım.
db = FAISS.from_documents(chunks, embeddings)

# 3. Erişim (retrieval) fonksiyonu oluşturma

---



In [ ]:
# Benzerlik algoritmasının doğru sonuç verip vermediğini test edelim.
query = "RAG ile Kişisel Asistan eğitimi"
docs = db.similarity_search(query)
docs[0]

In [ ]:
# Kullanıcının girdileri ile arama yapmak için benzerlik algoritmasını kullanarak
# bir soru cevap zinciri oluşturuyoruz. (Kullanıcının girdisine artık verilen bağlama bakarak cevap verecek.)

chain = load_qa_chain(
    ChatGoogleGenerativeAI(
        temperature=0.7,
        api_key=os.environ.get("GEMINI_API_KEY"),
        model="gemini-1.5-flash"),
    chain_type="stuff"
)

In [ ]:
# Oluşturduğumuz soru cevap zincirini çalıştırarak dokümanlar üzerinde test edelim.
query = "RAG ile Kişisel Asistan eğitimini kim veriyor?"
docs = db.similarity_search(query)

chain.run(input_documents=docs, question=query)

# 4. Sohbet hafızalı bir sohbet botu oluşturma

---



In [17]:
from IPython.display import display
import ipywidgets as widgets

# Daha önce FAISS kullanarak oluşturduğumuz vektör veritabanını
# dokümanlarımıza erişim mekanizması olarak kullanarak
# hafızaya sahip olan (önceki mesajları hatırlayabilen) bir konuşma zinciri oluşturalım.
qa = ConversationalRetrievalChain.from_llm(ChatGoogleGenerativeAI(temperature=0.7, api_key=os.environ.get("GEMINI_API_KEY"), model="gemini-1.5-flash"), db.as_retriever())

In [ ]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("GDG On Campus Trakya Chatbot'unu kullandığınız için teşekkürler!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("GDG On Campus Trakya Chatbot'una hoşgeldiniz!")

input_box = widgets.Text(placeholder='Lütfen sorunuzu girin:')
input_box.on_submit(on_submit)

display(input_box)

# 5. Tüm işlemi Gradio ile daha kullanılabilir hale getirme

---



In [ ]:
# Token sayma fonksiyonu
tokenizer = tokenization.get_tokenizer_for_model("gemini-1.5-flash")

def count_tokens(text: str) -> int:
    result = tokenizer.count_tokens(text)
    return result.total_tokens


# Chatbot fonksiyonu
def chatbot(api_key, uploaded_file, chat_history, query):
    os.environ["GEMINI_API_KEY"] = api_key

    # PDF'i hafızaya yüklüyoruz
    loader = PyPDFLoader(uploaded_file.name)
    pages = loader.load_and_split()

    # Metni işliyoruz
    doc = textract.process(uploaded_file.name)
    text = doc.decode('utf-8')

    # Metinleri parçalara ayırma
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=24,
        length_function=count_tokens,
    )
    chunks = text_splitter.create_documents([text])

    # Embedding işlemi
    embeddings = GoogleGenerativeAIEmbeddings(
        google_api_key=os.environ.get("GEMINI_API_KEY"),
        model="models/embedding-001"
    )

    # Vektör veritabanı oluşturma
    db = FAISS.from_documents(chunks, embeddings)

    # Konuşma zinciri oluşturma
    qa = ConversationalRetrievalChain.from_llm(
        ChatGoogleGenerativeAI(
            temperature=0.7,
            api_key=os.environ.get("GEMINI_API_KEY"),
            model="gemini-1.5-flash"
        ),
        db.as_retriever()
    )

    result = qa({"question": query, "chat_history": chat_history})
    answer = result['answer']
    chat_history.append((query, answer))
    return chat_history, chat_history

# Gradio arayüzü
with gr.Blocks() as demo:
    gr.Markdown("# 🦜🔗LangChain ile Kişiye Özel Chatbot - PDF'lerinizle Konuşun!")

    with gr.Column():
        api_key = gr.Textbox(label="Lütfen GEMINI API anahtarınızı girin:", type="password")
        uploaded_file = gr.File(label="Lütfen bir PDF dosyası yükleyin", file_types=[".pdf"])
        chatbot_interface = gr.Chatbot()
        query = gr.Textbox(label="Lütfen sorunuzu girin:")

    chat_history = gr.State([])

    def respond(api_key, uploaded_file, chat_history, query):
        chat_history, _ = chatbot(api_key, uploaded_file, chat_history, query)
        return chat_history, ""

    query.submit(respond, [api_key, uploaded_file, chat_history, query], [chatbot_interface, query])

demo.launch()
